# Professional Athlete Classifier Using Azure Machine Learning Service 

The following tutorial demonstrates an end-to-end workflow for training and deploying a **Tensorflow 2.0** model on Azure Machine Learning Service. The notebook will cover the following topics:

- Azure Machine Learning service workspace and basic concepts
- Creating a dataset using Azure Cognitive Services and hosting it on a blob datastore
- Setting up an auto-scaling AML Compute cluster
- Training a distributed model across multiple nodes and GPUs
- Deploying a model and creating a web service endpoint


## Problem Statement

Professional sporting events have the power to bring friends, families, and co-workers together at home, at bars, and at the venue itself. While most people enjoy the social aspect and the entertainment of the game, few can recognize every single athlete that is on the court, field, or ice.

In this demo, we will be building a model that help solves this problem by classifying NBA players based upon their photo. While this can be extended to a multi-classification problem, this example will focus on a binary classification between two of the top NBA players today - Lebron James and Stephen Curry.

<table>
    <tr>
        <td>
            <h2>Lebron James</h2>
            <img src='./test/lebron.jpg' height='200' width='400'>
        </td>
        <td>
            <h2>Stephen Curry</h2>
            <img src='./test/steph.jpg' height='300' width='475'>
        </td>
    </tr>
</table>

## What is Azure Machine Learning Service?
Azure Machine Learning service is a cloud service that you can use to develop and deploy machine learning models. Using Azure Machine Learning service, you can track your models as you build, train, deploy, and manage them, all at the broad scale that the cloud provides.
![](./images/aml-overview.png)

## Prerequisites

The tutorial requires the following prerequisites:

- An existing Azure Machine Learning service workspace
- An existing notebook VM instance inside the workspace

The tutorial is designed to be run on the notebook VM instance. Click [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/tutorial-1st-experiment-sdk-setup) for instructions on creating a workspace and notebook VM.

## Connect To Workspace

The [workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace(class)?view=azure-ml-py) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. The workspace holds all your experiments, compute targets, models, datastores, etc.

You can [open ml.azure.com](https://ml.azure.com) to access your workspace resources through a graphical user interface of **Azure Machine Learning studio**.

![](./images/aml-workspace.png)

**You will be asked to login in the next step. Use your Microsoft AAD credentials.**

In [109]:
from azureml.core import Workspace

workspace = Workspace.from_config()
print('Workspace name: ' + workspace.name, 
      'Azure region: ' + workspace.location, 
      'Subscription id: ' + workspace.subscription_id, 
      'Resource group: ' + workspace.resource_group, sep = '\n')

Workspace name: john-east
Azure region: eastus
Subscription id: 15ae9cb6-95c1-483d-a0e3-b1a1a3b06324
Resource group: john


## Create Dataset Using Azure Cognitive Services

[Azure Cognitive Services](https://azure.microsoft.com/en-us/services/cognitive-services/) is a comprehensive family of AI services and cognitive APIs. We will be using the [Bing Image Search API](https://azure.microsoft.com/en-us/services/cognitive-services/bing-image-search-api/) to find and download images of professional athletes in order to build a dataset that can be used for training.

![](./images/cog-services.png)

To complete this section, you will need to create an Azure Cognitive Service resource in order to obtain an API key. [Click here](https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-apis-create-account?tabs=multiservice%2Cwindows) for instructions on how to do so.

**Save the API key in a variable that we will use later.**

In [116]:
cognitive_service_api_key = '<API-KEY-HERE>'

We have prepared a script which does the following steps in sequence:

1. For each athlete, make a query to the Bing Image Search API for profile photos.
2. Download each image from the resulting response and crop out the face of the athlete.
3. Save the images to a data directory in corresponding test/valid and label folders.

**Let's take a look at the data preparation script**

In [118]:
%pycat src/cognitive-service-web-scraper.py

**Install the necessary packages to the run the script**

In [119]:
%pip install opencv-python imutils

Note: you may need to restart the kernel to use updated packages.


The script downloads images in a predefined directory. In order to make it easier for training later on, we will mount a [Blob Storage](https://azure.microsoft.com/en-us/services/storage/blobs/) container into our local directory and download the images directly into that datastore.

When an Azure Machine Learning workspace is created, a datastore is created by default.

**Get the default datastore of the workspace**

In [121]:
# Get default datastore
datastore = workspace.get_default_datastore()

**Create a blobfuse config file which is required for mounting**

In [ ]:
config_file_name = 'blobfuse_connection.cfg'

config_params = 'accountName {}\naccountKey {}\ncontainerName {}'.format(
    datastore.account_name, 
    datastore.account_key, 
    datastore.container_name)

config_file = open(config_file_name, 'w')
config_file.writelines(config_params)
config_file.close()

**Create directory that will map to the Blob Storage container**

In [14]:
blobstore_dir = 'blobstore'

In [ ]:
import os
if not os.path.exists(blobstore_dir):
    os.makedirs(blobstore_dir)

**Mount the datastore using blobfuse**

In [15]:
!sudo blobfuse {blobstore_dir} --tmp-path=/mnt/resource/blobfusetmp  --config-file={config_file_name} -o attr_timeout=240 -o entry_timeout=240 -o negative_timeout=120

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


**Now we can run the script to build our dataset**

In [6]:
data_dir = 'nba-players'

In [17]:
!sudo mkdir {blobstore_dir}/{data_dir}

In [18]:
import sys
!sudo {sys.executable} cognitive-service-web-scraper.py --root_dir {blobstore_dir}/{data_dir} --image_dim 400 --api_key {cognitive_service_api_key}

Writing image 0 for Lebron James...
Writing image 1 for Lebron James...
Writing image 2 for Lebron James...
Writing image 3 for Lebron James...
Writing image 4 for Lebron James...
Writing image 5 for Lebron James...
Writing image 6 for Lebron James...
Writing image 7 for Lebron James...
Writing image 8 for Lebron James...
Writing image 9 for Lebron James...
Writing image 10 for Lebron James...
Writing image 11 for Lebron James...
Writing image 12 for Lebron James...
Writing image 13 for Lebron James...
Writing image 14 for Lebron James...
Writing image 15 for Lebron James...
Writing image 16 for Lebron James...
Writing image 17 for Lebron James...
Writing image 18 for Lebron James...
Writing image 19 for Lebron James...
Writing image 20 for Lebron James...
Writing image 21 for Lebron James...
Writing image 22 for Lebron James...
Writing image 23 for Lebron James...
Writing image 24 for Lebron James...
Writing image 25 for Lebron James...
Writing image 26 for Lebron James...
Writing ima

Writing image 210 for Lebron James...
Writing image 211 for Lebron James...
Writing image 212 for Lebron James...
Writing image 213 for Lebron James...
Writing image 214 for Lebron James...
Writing image 215 for Lebron James...
Writing image 216 for Lebron James...
Writing image 217 for Lebron James...
Writing image 218 for Lebron James...
Writing image 219 for Lebron James...
Writing image 220 for Lebron James...
Writing image 221 for Lebron James...
Writing image 222 for Lebron James...
Writing image 223 for Lebron James...
Writing image 224 for Lebron James...
Writing image 225 for Lebron James...
Writing image 226 for Lebron James...
Writing image 227 for Lebron James...
Writing image 228 for Lebron James...
Writing image 229 for Lebron James...
Writing image 230 for Lebron James...
Writing image 231 for Lebron James...
Writing image 232 for Lebron James...
Writing image 233 for Lebron James...
Writing image 234 for Lebron James...
Writing image 235 for Lebron James...
Writing imag

Writing image 420 for Lebron James...
Writing image 421 for Lebron James...
Writing image 422 for Lebron James...
Writing image 423 for Lebron James...
Skipping blobstore/nba-players/valid/Lebron_James/423.jpg due to download error:
Writing image 423 for Lebron James...
Writing image 424 for Lebron James...
Writing image 425 for Lebron James...
Writing image 426 for Lebron James...
Writing image 427 for Lebron James...
Writing image 428 for Lebron James...
Writing image 429 for Lebron James...
Writing image 430 for Lebron James...
Writing image 431 for Lebron James...
Writing image 432 for Lebron James...
Writing image 433 for Lebron James...
Writing image 434 for Lebron James...
Writing image 435 for Lebron James...
Writing image 436 for Lebron James...
Writing image 437 for Lebron James...
Writing image 438 for Lebron James...
Writing image 439 for Lebron James...
Writing image 440 for Lebron James...
Writing image 441 for Lebron James...
Writing image 442 for Lebron James...
Writing

Writing image 128 for Stephen Curry...
Writing image 129 for Stephen Curry...
Writing image 130 for Stephen Curry...
Writing image 131 for Stephen Curry...
Writing image 132 for Stephen Curry...
Writing image 133 for Stephen Curry...
Writing image 134 for Stephen Curry...
Writing image 135 for Stephen Curry...
Writing image 136 for Stephen Curry...
Writing image 137 for Stephen Curry...
Writing image 138 for Stephen Curry...
Writing image 139 for Stephen Curry...
Writing image 140 for Stephen Curry...
Writing image 141 for Stephen Curry...
Writing image 142 for Stephen Curry...
Writing image 143 for Stephen Curry...
Writing image 144 for Stephen Curry...
Writing image 145 for Stephen Curry...
Writing image 146 for Stephen Curry...
Writing image 147 for Stephen Curry...
Writing image 148 for Stephen Curry...
Writing image 149 for Stephen Curry...
Writing image 150 for Stephen Curry...
Writing image 151 for Stephen Curry...
Writing image 152 for Stephen Curry...
Writing image 153 for Ste

Writing image 329 for Stephen Curry...
Writing image 330 for Stephen Curry...
Writing image 331 for Stephen Curry...
Writing image 332 for Stephen Curry...
Writing image 333 for Stephen Curry...
Writing image 334 for Stephen Curry...
Writing image 335 for Stephen Curry...
Writing image 336 for Stephen Curry...
Writing image 337 for Stephen Curry...
Writing image 338 for Stephen Curry...
Writing image 339 for Stephen Curry...
Writing image 340 for Stephen Curry...
Writing image 341 for Stephen Curry...
Writing image 342 for Stephen Curry...
Writing image 343 for Stephen Curry...
Writing image 344 for Stephen Curry...
Writing image 345 for Stephen Curry...
Writing image 346 for Stephen Curry...
Writing image 347 for Stephen Curry...
Writing image 348 for Stephen Curry...
Writing image 349 for Stephen Curry...
Writing image 350 for Stephen Curry...
Writing image 351 for Stephen Curry...
Writing image 352 for Stephen Curry...
Writing image 353 for Stephen Curry...
Writing image 354 for Ste

...

Cropping image: blobstore/nba-players/train/Stephen_Curry/81.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/338.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/182.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/203.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/181.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/45.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/0.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/132.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/340.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/29.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/113.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/139.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/187.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/150.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/34.jpg
Cropping image: 

Cropping image: blobstore/nba-players/train/Stephen_Curry/1.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/248.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/97.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/274.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/307.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/329.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/75.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/328.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/111.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/298.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/229.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/211.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/215.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/228.jpg
Cropping image: blobstore/nba-players/train/Stephen_Curry/308.jpg
Cropping image

Cropping image: blobstore/nba-players/train/Lebron_James/178.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/179.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/18.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/180.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/181.jpg
Removing corrupted file: blobstore/nba-players/train/Lebron_James/181.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/182.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/183.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/184.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/185.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/186.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/187.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/188.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/189.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/19.jpg
Cropping image: bl

Cropping image: blobstore/nba-players/train/Lebron_James/288.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/289.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/29.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/290.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/291.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/292.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/293.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/294.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/295.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/296.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/297.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/298.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/299.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/3.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/30.jpg
Cropping image: blobstore/nba

Cropping image: blobstore/nba-players/train/Lebron_James/86.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/87.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/88.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/89.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/9.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/90.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/91.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/92.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/93.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/94.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/95.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/96.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/97.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/98.jpg
Cropping image: blobstore/nba-players/train/Lebron_James/99.jpg


## Register Dataset

Now we can register the directory in our datastore as a [Dataset](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.dataset.dataset?view=azure-ml-py), which is simply a reference to a file or folder in a storage container. Registered datasets can then be mounted to compute targets during training making it very easy to have your data be accessible on any machine during training.

In [7]:
from azureml.core import Dataset

nba_players_dataset = Dataset.File.from_files(path=(datastore, data_dir))

nba_players_dataset = nba_players_dataset.register(workspace=workspace,
                                       name='NBA Players Dataset',
                                       description='Dataset containing pictures of NBA players')

**If the dataset has already been registered, then you (and other users in your workspace) can directly run this cell to get access to it.**

In [8]:
nba_players_dataset = workspace.datasets['NBA Players Dataset']

## Create Compute Target

A [compute target](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.computetarget?view=azure-ml-py) is a designated compute resource/environment where you run your training script or host your service deployment. This location may be your local machine or a cloud-based compute resource. Compute targets can be reused across the workspace for different runs and experiments. 

For this tutorial, we will create an auto-scaling [Azure Machine Learning Compute](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute?view=azure-ml-py) cluster, which is a managed-compute infrastructure that allows the user to easily create a single or multi-node compute. To create the cluster, we need to specify the following parameters:

- `vm_size`: The is the type of GPUs that we want to use in our cluster. For this tutorial, we will use **Standard_NC12 (NVIDIA K80) GPU Machines** .
- `idle_seconds_before_scaledown`: This is the number of seconds before a node will scale down in our auto-scaling cluster. We will set this to **6000** seconds. 
- `min_nodes`: This is the minimum numbers of nodes that the cluster will have. To avoid paying for compute while they are not being used, we will set this to **0** nodes.
- `max_modes`: This is the maximum number of nodes that the cluster will scale up to. Will will set this to **4** nodes.

**When jobs are submitted to the cluster it takes approximately 5 minutes to allocate new nodes** 

In [97]:
from azureml.core.compute import AmlCompute, ComputeTarget

cluster_name = 'k80cluster'
compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC6',
                                                       idle_seconds_before_scaledown=1200,
                                                       min_nodes=0, 
                                                       max_nodes=2)

compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


**If the compute target has already been created, then you (and other users in your workspace) can directly run this cell to get access to it.**

In [98]:
compute_target = workspace.compute_targets['k80cluster']

## Perform Experiment

Now that we have our dataset registered and compute target created, it is time to start training our model. We will start by creating an [experiment](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.experiment.experiment?view=azure-ml-py). An experiment is a grouping of many runs from a specified script. All runs in this tutorial will be performed under the same experiment. 

In [99]:
from azureml.core import Experiment

experiment_name = 'nba-player-classifier' 
experiment = Experiment(workspace, name=experiment_name)

A training script has been prepared in advance and will be used to train our model. The Tensorflow 2.0 training script adds dropout and classification layers to a base ResNet50 model and then fine-tunes the weights on our new dataset.

**Let's take a look at what the script looks like exactly**

In [126]:
%pycat src/train.py

**Next step is to create a TensorFlow Estimator**

The Azure Machine Learning Python SDK Estimator classes allow you to easily construct run configurations for your experiments. They allow you too define parameters such as the training script to run, the compute target to run it on, framework versions, additional package requirements, etc. 

You can also use a generic [Estimator](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.estimator.estimator?view=azure-ml-py) to submit training scripts that use any learning framework you choose.

For popular libaries like PyTorch and Tensorflow you can use their framework specific estimators. We will use the [TensorFlow Estimator](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.dnn.tensorflow?view=azure-ml-py) for our experiment.

In [124]:
from azureml.train.dnn import TensorFlow

estimator = TensorFlow(source_directory='src',
                                  entry_script='train.py',
                                  compute_target=compute_target,
                                  script_params = {
                                      '--data_dir': nba_players_dataset.as_named_input('nbaplayersdata').as_mount(),
                                      '--image_dim': 250,
                                      '--learning_rate': 0.001,
                                      '--batch_size': 16,
                                      '--steps_per_epoch': 100,
                                      '--num_epochs': 25,
                                      '--dropout_rate': 0.5,
                                      '--activation_function': 'softmax',
                                      '--output_dir':'./outputs',
                                  },
                                  framework_version='2.0',
                                  use_gpu=True,
                                  pip_packages=['Pillow==6.2.0', 'scipy==1.1.0', 'azureml-dataprep[fuse,pandas]==1.1.29'])

**Now we submit our estimator to the experiment**

In [ ]:
run = experiment.submit(estimator)

**We can monitor the progress of our run within the notebook**

In [146]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

**Now we can register the trained model into our workspace**

In [152]:
model = run.register_model(model_name='nba-player-classifier', 
                           model_path='outputs',
                           datasets=[('train, test, validation data', nba_players_dataset)],
                           description='ResNet 50 model of for classifying NBA players')

## Deploy Model

Once we are satisified with the performance of our model, we can deploy it as a web service. For this tutorial, we will be deploying our service to an [Azure Container Instance (ACI)](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.aciwebservice?view=azure-ml-py). Azure Machine Learning has options for [local](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.localwebservice?view=azure-ml-py) and [Azure Kubernetes Service (AKS)](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.akswebservice?view=azure-ml-py) deployment as well. Deployment of a model requires the following dependencies:

- A scoring script to show how to use the model
- An environment file to show what packages need to be installed
- A configuration file to build the web service
- The model that was previously trained

The scoring script requires two methods:

- **init()**: This method is run upon deployment of the web service container. This is typically where the model is loaded.
- **run()**: This method is run each time a request is sent to the web service. This is your predict method.

**Let's take a look at the scoring script that has already been created.**

In [150]:
%pycat src/score.py

To assist with creating the environment file, we will be using the [CondaDepencies](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.conda_dependencies.condadependencies?view=azure-ml-py) class to define the required python packages for running our scoring script. 

**The code below creates the environment file that we need for deployment.**

In [153]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(conda_packages=['numpy','pandas'],
                                 pip_packages=['inference-schema[numpy-support]', 'azureml-defaults', 'tensorflow==2.0.0', 'opencv-python==4.1.1.26'])

with open('src/myenv.yml','w') as f:
    f.write(myenv.serialize_to_string())

**We define our deployment configuration using the [InferenceConfig](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.inferenceconfig?view=azure-ml-py) class.**

In [154]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(source_directory='src',
                                   runtime= 'python', 
                                   entry_script='score.py',
                                   conda_file='myenv.yml')

**Now we can create and deploy our web service (deployment will take about 10 minutes).**

In [155]:
from azureml.core.model import Model
from azureml.core.webservice import Webservice
from azureml.exceptions import WebserviceException
from azureml.core.webservice import AciWebservice, Webservice

aci_config = AciWebservice.deploy_configuration(cpu_cores=2, 
                                                memory_gb=4, 
                                                tags={'model': 'RESNET 50', 'method': 'tensorflow'}, 
                                                description='Predicts NBA player based on image')

aci_service_name = 'nba-player-aciservice'

try:
    aci_service = Webservice(workspace, name=aci_service_name)
    if aci_service:
        aci_service.delete()
except WebserviceException as e:
    print()

In [ ]:
aci_service = Model.deploy(workspace, aci_service_name, [model], inference_config, aci_config)

aci_service.wait_for_deployment(True)
print(aci_service.state)

Running...............................................................................................................................................................................................................................................................

In [ ]:
print(aci_service.scoring_uri)

In [ ]:
import requests
image_url = 'https://a.espncdn.com/photo/2019/0603/r551314_1296x729_16-9.jpg' #Steph Curry
input_data = '{\"image_url\": \"'+ image_url +'\"}'
headers = {'Content-Type':'application/json'}
response = requests.post(aci_service.scoring_uri, input_data, headers=headers)

print(response.text)